# Problem 1: Decision Trees (DTs) and Random Forests
In this question, you are asked to build a decision tree from scratch. You can use evaluation metrics from sklearn, but cannot use the decision tree package from sklearn.

Decision trees consists of nodes, each containing a set of data samples. 
During training, the child nodes are created by splitting the node's data samples based on the values of a certain feature.
During testing, the test data sample is passed through the tree until it hits a leaf node, where its label is predicted as the label of the majority of the data samples in that leaf node.

## Load the Apple Quality Dataset

We will use the apple quality dataset for this problem. 
Load the dataset (given in `data/archive.zip`) into two numpy arrays X and Y.

- X should be a 4000*7 numpy array (4000 apples and 7 attributes: size, weight, sweetness, crunshiness, juiciness, ripeness, acidity)
- Y should be a 4000*1 numpy array (with value 1 for good quality and 0 for bad quality)

In [303]:
import os
import pandas as pd
import numpy as np

IS_TEST_MODE = False

zip_file_name = "apple_quality.zip"
csv_file_name = "apple_quality.csv"
zip_file_path = f"data/{zip_file_name}"

if IS_TEST_MODE:
    import zipfile

    num_rows = 10000
    temp_path = "/tmp"

    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extract(member=csv_file_name, path=temp_path)
        csv_file_path = os.path.join(temp_path, csv_file_name)

        df = pd.read_csv(csv_file_path, nrows=num_rows)

else:
    dataset_path = os.path.abspath(zip_file_path)
    df = pd.read_csv(dataset_path)

# Remove the last row
df = df.iloc[:-1]
df[["Acidity"]] = df[["Acidity"]].apply(pd.to_numeric, errors="coerce")

# Create the arrays X, Y from the dataframe
# ===== YOUR CODE HERE =====
X = df.iloc[:, 1:8]
Y = df.iloc[:, -1]

df
# ==========================

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good
...,...,...,...,...,...,...,...,...,...
3995,3995.0,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,bad
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,good
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,bad
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,good


Split the data into training and test data with a ratio of 0.3 (2800 data samples in the training set and 1200 data samples in the test set):

- X_train should be a 2800*7 numpy array
- Y_train should be a 2800*1 numpy array
- X_test should be a 1200*7 numpy array
- Y_test should be a 1200*1 numpy array

In [304]:
# ===== YOUR CODE HERE =====
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).reshape(-1, 1)
Y_test = np.array(Y_test).reshape(-1, 1)

X
# ==========================

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984
...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720


## Quality metrics

Splitting a node when training a DT is done based on certain quality metrics.
There are multiple popular quality metrics, but for this assignment, we only consider **information gain** and **Gini impurity**.

### Information gain 

Information gain is the difference between the entropy of the parent node and the weighted average of the entropy of child nodes.

Entropy is given by the following formula:

$E=-\Sigma_{i=1}p_i \log_2 p_i$

where $p_i$ is the class probability, i.e., out of all samples in the node, what fraction belongs to the class $i$.
Lower entropy implies that the classes are well divided, which is our goal. 
Hence, we try to maximize the information gain.

### Gini impurity

Gini impurity is given by the following formula:

$G = \Sigma_{i=1}p_i(1-p_i)=1-\Sigma_{i=1}p_i^2$

where $p_i$ is once again the class probability.
Notice that lower Gini impurity implies that the classes are well divided. 
Hence, we try to maximize the Gini impurity decrease, i.e., the difference between Gini impurity of the parent node and weighted average of the Gini impurity of the child nodes.

For simplicity in implementation, we will use the same function for both information gain and Gini impurity, using the entropy or the Gini impurity based on an argument. 
Assume that the weights for taking the weighted average is the ratio of the number of data samples in the corresponding child node and the number of data samples in the parent node.

The basic structure of code is shown below and you need to finish the code according to the structure. 
Do not delete/change the names of the pre-defined functions since they are used in grading. 
You can add functions as you need. 
You can only use numpy and math library to implement these two metrics.

In [305]:
def entropy(labels_list: np.array) -> float:
    """Returns the entropy given a list of labels.

    Args:
        labels_list: Numpy array containing labels.

    Returns:
        Entropy.
    """
    # ===== YOUR CODE HERE =====
    _, counts = np.unique(labels_list, return_counts=True)
    total = sum(counts)
    probs = [count / total for count in counts]
    
    entropy_score = 0.0
    for prob in probs:
        entropy_score += prob * np.log2(prob)
    
    return -1.0 * entropy_score
    # ==========================


def gini_impurity(labels_list: np.array) -> float:
    """Returns the Gini impurity given a list of labels.

    Args:
        labels_list: Numpy array containing labels.

    Returns:
        Gini impurity.
    """
    # ===== YOUR CODE HERE =====
    _, counts = np.unique(labels_list, return_counts=True)
    total = sum(counts)
    probs = [count / total for count in counts]

    gini_purity = 0.0
    for prob in probs:
        gini_purity += prob**2
    
    return 1.0 - gini_purity
    # ==========================


def information_gain(
    parent_labels: np.array,
    l_child_labels: np.array,
    r_child_labels: np.array,
    mode: str,
) -> float:
    """Returns the information gain or the Gini impurity decrease given a list of labels.

    Args:
        parent_labels: Numpy array containing labels in the parent node.
        l_child_labels: Numpy array containing labels in the left child node.
        r_child_labels: Numpy array containing labels in the right child node.
        mode: Either "entropy" or "gini" to select between entropy or Gini impurity.

    Returns:
        Information gain or the Gini impurity decrease based on the mode.
    """
    # ===== YOUR CODE HERE =====
    def calc_information_gain(
        parent_labels: np.array,
        l_child_labels: np.array,
        r_child_labels: np.array,
        info_function: callable
    ) -> float:
        parent_metric = info_function(parent_labels)
        l_child_metric = info_function(l_child_labels)
        r_child_metric = info_function(r_child_labels)

        weighted_children_metrics = (
            (l_child_metric * len(l_child_labels) / len(parent_labels)) +
            (r_child_metric * len(r_child_labels) / len(parent_labels))
        )

        return parent_metric - weighted_children_metrics

    if mode == "entropy":
        return calc_information_gain(parent_labels, l_child_labels, r_child_labels, entropy)
    
    elif mode == "gini":
        return calc_information_gain(parent_labels, l_child_labels, r_child_labels, gini_impurity)
    
    else:
        raise Exception(f"Unknown mode: {mode}")
    # ==========================

## Finding the best split 

During training, a leaf node in the decision tree can be split based on any of the input features at any threshold.
However, we want to find the best split in terms of the quality metrics defined above (highest information gain or Gini impurity decrease).

For this assignment, let us use a brute force method and loop through all features and all possible thresholds to find the best possible split.

Fill the code below to return the best split in the form of a dictionary (see the code comments for the format).

In [306]:
def find_best_split(dataset: np.array, num_features: int, split_mode: str) -> dict:
    """Find the best split according to the given quality metric.

    Args:
        dataset: Numpy array of shape (N, k + 1), where N is the number of data samples
            and k is the number of features. First k columns contain the features and the
            last column contains the label.
        num_features: Number of features.
        split_mode: Either "entropy" or "gini" to select between entropy or Gini impurity
            while calculating the quality metric.

    Returns:
        Dict containing the best split. It should have the following keys:
            - "feature_index": Index of the feature to split on.
            - "threshold": Threhold of the feature value to split on.
            - "dataset_left": Left side of the split dataset (feature value < threshold).
            - "dataset_right": Right side of the split dataset (feature value >=
                threshold).
            - "info_gain": Information gain or Gini impurity decrease (select the mode
                based on the value of split_mode).
    """

    def _split(data: np.array, feature_idx: int, feature_threshold: float) -> tuple:
        """Splits the data into two sets based on the value of the feature at feature_idx
        and the threshold given by feature_threshold."""

        mask_below_threshold = data[:, feature_idx] < feature_threshold
        group1 = data[mask_below_threshold]
        group2 = data[~mask_below_threshold]

        return group1, group2

    best_split = {
        "feature_index": None,
        "threshold": None,
        "dataset_left": None,
        "dataset_right": None,
        "info_gain": 0,
    }
    max_info_gain = -float("inf")

    # Loop over all the features.
    for feature_index in range(num_features):
        feature_values = dataset[:, feature_index]
        possible_threshold = np.unique(feature_values)

        # Loop over all the feature values present in the data.
        for threshold in possible_threshold:
            ...
            # 1. Get the current split.
            # 2. Ensure that both splits are not empty.
            # 3. Compute information gain.
            # 4. Update the best split if information gain is higher.

            # ===== YOUR CODE HERE =====
            l_child_dataset, r_child_dataset = _split(dataset, feature_index, threshold)
            if l_child_dataset.size == 0 or r_child_dataset.size == 0:
                continue

            parent_labels = dataset[:, num_features]
            l_child_labels = l_child_dataset[:, num_features]
            r_child_labels = r_child_dataset[:, num_features]
            
            info_gain = information_gain(parent_labels, l_child_labels, r_child_labels, split_mode)
            if info_gain > max_info_gain:
                best_split = {
                    "feature_index": feature_index,
                    "threshold": threshold,
                    "dataset_left": l_child_dataset,
                    "dataset_right": r_child_dataset,
                    "info_gain": info_gain,
                }
                max_info_gain = info_gain
            # ==========================

    # Return best split
    return best_split

## Building the tree

Finally, we recursively build the tree starting from the root:

- If the stopping conditions are not met, find the best split at the root and recursively call the `build_tree` function from the left and the right child nodes.

- Keep splitting until the stopping conditions are met. In this assignment, we have two stopping conditions: the depth of the tree reaches `MAX_DEPTH` or the number of data samples in the current node is less than `MIN_SAMPLE_SPLIT`. You can adjust these two parameters.

The tree should use the `Node` class given below. 
Leaf nodes should at least have the `value` field filled, while decision nodes should have all fields except `value` filled (see the code comments for what they should contain). 
The `build_tree` function should return the filled root (when called recursively, root here is the root of the sub-tree) node of the decision tree. 

In [307]:
from typing_extensions import Self


class Node:
    """Node class for decision tree.

    Args:
        feature_index: (only required for decision nodes) Index of the best split feature.
        feature_threshold: (only required for decision nodes) Threshold value of the best
            split feature.
        left: (only required for decision nodes) Left child node.
        right: (only required for decision nodes) Right child node.
        info_gain: (only required for decision nodes) The information gain obtained by the
            best split on this node.
        value: (only required for leaf nodes) The class label to predict if a test sample
            reaches this leaf node (equal to the class label of the majority of the data
            samples in this leaf node).
    """

    def __init__(
        self,
        feature_index: int = None,
        feature_threshold: float = None,
        left: Self = None,
        right: Self = None,
        info_gain: float = None,
        value: int = None,
    ):
        # For decision node
        self.feature_index = feature_index
        self.threshold = feature_threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # For leaf node
        self.value = value


# Hyperparameters.
MAX_DEPTH = 100
MIN_SAMPLE_SPLIT = 3


def build_tree(dataset: np.array, split_mode: str, curr_depth: int = 0) -> Node:
    """Build a decision tree given a training dataset.

    Args:
        dataset: Numpy array of shape (N, k + 1), where N is the number of data samples
            and k is the number of features. First k columns contain the features and the
            last column contains the label.
        split_mode: Either "entropy" or "gini" to select between entropy or Gini impurity
            while calculating the quality metric.
        curr_depth: Current depth of the tree (starts from 0 and goes up by 1 when
            recursively calling this function on child nodes).

    Returns:
        The root node of the decision tree (in the form of a Node class).
    """

    def calculate_leaf_value(Y: np.array) -> int:
        """Function to compute the value of leaf node.

        Args:
            Y: Array of labels.
        """
        Y = list(Y)
        return max(Y, key=Y.count)

    # 1. Preprocess the dataset if needed
    # 2. Split until stopping conditions are met
    # 2.1 Find the best split
    # 2.2 If information gain is positive,
    # 2.2.1 Recur left
    # 2.2.2 Recur right
    # 2.2.3 Return the filled decision node (in the form of a Node class)

    # 3. If the function reaches this point, it means that this node is a leaf node.
    #    Calculate the value.
    # 4. Return the filled leaf node (in the form of a Node class)

    # ===== YOUR CODE HERE =====
    NUM_FEATURES = 7
    labels = dataset[:, NUM_FEATURES]

    if curr_depth > MAX_DEPTH or len(dataset) < MIN_SAMPLE_SPLIT:
        return Node(value=calculate_leaf_value(labels))
    
    def extract_dict(
        feature_index: int,
        threshold: float,
        dataset_left: np.array,
        dataset_right: np.array,
        info_gain: float,
    ) -> tuple:
        return feature_index, threshold, dataset_left, dataset_right, info_gain
    
    best_split = find_best_split(dataset, NUM_FEATURES, split_mode)
    feature_index, threshold, dataset_left, dataset_right, info_gain = extract_dict(**best_split)

    if info_gain > 0:
        return Node(
            feature_index=feature_index,
            feature_threshold=threshold,
            left=build_tree(dataset_left, split_mode, curr_depth=(curr_depth + 1)),
            right=build_tree(dataset_right, split_mode, curr_depth=(curr_depth + 1)),
            info_gain=info_gain,
        )

    else:
        return Node(value=calculate_leaf_value(labels))
    # ==========================

## Training

The code below trains a decision tree using the `build_tree` function.

In [308]:
def train(X: np.array, Y: np.array, split_mode: str) -> Node:
    """Train a decision tree.

    Args:
        X: Array containing the training inputs.
        Y: Array containing the training labels.
        split_mode: Either "entropy" or "gini" to select between entropy or Gini impurity
            while calculating the quality metric.

    Returns:
        Root node of a trained decision tree.
    """
    dataset = np.concatenate((X, Y), axis=1)
    root = build_tree(dataset, split_mode)
    return root

## Prediction

`predict` function given below predicts the class labels of given test inputs.
Fill the `make_prediction` function, which recursively takes a single data sample down the decision tree and predicts the class label given by the leaf node it encounters.

In [309]:
def predict(X: np.array, tree: Node) -> np.array:
    """Predict the labels with a decision tree.

    Args:
        X: Array containing the test inputs.
        tree: The root node of a trained decision tree.

    Returns:
        The class labels corresponding to the test inputs.
    """
    predictions = np.array([make_prediction(x, tree) for x in X])
    return predictions


def make_prediction(x: np.array, tree: Node) -> int:
    """Predict the label of a single data point.

    Args:
        x: A single test input (array of size num_features)
        tree: The root node of a decision sub-tree (hint: you may need to call this
            function recursively to find the label)

    Returns:
        The class label corresponding to the test input.
    """
    ...

    # Hint: you need to recursively call this function based on the split feature and
    # threshold at the current node till you reach a leaf node.

    # ===== YOUR CODE HERE =====
    if tree.value is not None:
        return tree.value
    
    feature_value = x[tree.feature_index]
    if feature_value < tree.threshold:
        return make_prediction(x, tree.left)
    else:
        return make_prediction(x, tree.right)
    # ==========================

Print the following:

1. The tree structure (you can use the given `print_tree` function)
2. Accuracy and F1 score on test set for both information gain and gini impurity decrease. The format should be:
```    
Information gain
    accuracy
    F1_score
Gini impurity decrease
    accuracy
    F1_score
```
3. The final `max_depth` and `min_leaf_node` you use

In [310]:
def print_tree(tree: Node, indent: str = " "):
    """function to print the tree"""
    feature_map = {
        0: "Size",
        1: "Weight",
        2: "Sweetness",
        3: "Crunchiness",
        4: "Juiciness",
        5: "Ripeness",
        6: "Acidity",
    }
    if tree.value is not None:
        print(tree.value)

    else:
        print("X_" + feature_map[tree.feature_index], "<=", tree.threshold, "?")
        print("%sleft:" % (indent), end="")
        print_tree(tree.left, indent + indent)
        print("%sright:" % (indent), end="")
        print_tree(tree.right, indent + indent)


# Train the decision tree using `train` function and X_train, Y_train as the training
# dataset.
# Predict the labels for X_test and compute the accuracy, F1 score w.r.t. Y_test.
# Print the information asked in the above question.
# ===== YOUR CODE HERE =====
def calc_accuracy(Y_pred: np.array, Y_test: np.array):
    correct_predictions = sum(Y_pred == Y_test)[0]
    total_predictions = len(Y_test)

    return correct_predictions / total_predictions

def calc_f1_score(Y_pred: np.array, Y_test: np.array):
    true_positive = sum((Y_test == "good") & (Y_pred == "good"))[0]
    false_positive = sum((Y_test == "bad") & (Y_pred == "good"))[0]
    false_negative = sum((Y_test == "good") & (Y_pred == "bad"))[0]

    precision = (
        true_positive / (true_positive + false_positive) 
        if (true_positive + false_positive) > 0 else 0
    )
    recall = (
        true_positive / (true_positive + false_negative)
        if (true_positive + false_negative) > 0 else 0
    )
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0 else 0
    )

    if IS_TEST_MODE:
        print(
            f"Precision: {precision:.4f}\n"
            f"Recall: {recall:.4f}\n"
            f"F1 Score: {f1_score:.4f}"
        )

    return f1_score

def get_statistics(Y_pred: np.array, Y_test: np.array, split_mode: str) -> str:
    metric_title = "Information gain" if split_mode == "entropy" else "Gini impurity decrease"
    accuracy = calc_accuracy(Y_pred, Y_test)
    f1_score = calc_f1_score(Y_pred, Y_test)

    return (
        f"{metric_title}\n"
        f"  {accuracy:.4f}\n"
        f"  {f1_score:.4f}"
    )

In [311]:
# Expensive calculation 1

entropy_tree = train(X_train, Y_train, "entropy")
Y_pred_entropy = predict(X_test, entropy_tree).reshape(-1, 1)
entropy_statistics = get_statistics(Y_pred_entropy, Y_test, "entropy")

print_tree(entropy_tree)
print(entropy_statistics)

X_Juiciness <= -1.311596241 ?
 left:X_Weight <= 0.545296919 ?
  left:X_Crunchiness <= 0.080724492 ?
    left:X_Size <= -0.75474296 ?
        left:X_Sweetness <= 2.1913475 ?
                left:X_Weight <= 0.494272579 ?
                                left:bad
                                right:good
                right:X_Size <= -3.880292677 ?
                                left:bad
                                right:good
        right:X_Juiciness <= -2.473794792 ?
                left:X_Weight <= -1.841733528 ?
                                left:bad
                                right:X_Size <= 0.031080427 ?
                                                                left:bad
                                                                right:good
                right:good
    right:X_Size <= -0.559823724 ?
        left:X_Ripeness <= -0.208114246 ?
                left:X_Sweetness <= -2.329803018 ?
                                left:X_Size <= -0.865415634 ?
     

In [312]:
# Expensive calculation 2

gini_tree = train(X_train, Y_train, "gini")
Y_pred_gini = predict(X_test, gini_tree).reshape(-1, 1)
gini_statistics = get_statistics(Y_pred_gini, Y_test, "gini")

print_tree(gini_tree)
print(gini_statistics)

X_Juiciness <= -1.311596241 ?
 left:X_Weight <= 0.645451496 ?
  left:X_Crunchiness <= 0.080724492 ?
    left:X_Size <= -0.75474296 ?
        left:X_Sweetness <= 2.1913475 ?
                left:X_Weight <= 0.494272579 ?
                                left:bad
                                right:good
                right:X_Size <= -3.880292677 ?
                                left:bad
                                right:good
        right:X_Weight <= -2.570055123 ?
                left:bad
                right:X_Juiciness <= -2.473794792 ?
                                left:X_Juiciness <= -3.462401397 ?
                                                                left:good
                                                                right:X_Sweetness <= 1.847419978 ?
                                                                                                                                left:bad
                                                                      

In [313]:
# ==========================

# Random Forest

Decision trees (DTs) can be prone to overfitting, especially when they are deep. To mitigate this, we can use an ensemble method where we combine the predictions of multiple decisions trees and take an average or do majority voting to get our final prediction.

We will assume there are $m$ DTs and train them independently, similar to the previous problem. A popular method to train base learners in an ensemble method is via bagging ([bootstrap aggregating](https://en.wikipedia.org/wiki/Bootstrap_aggregating)). Given a training dataset of size $N$, bagging generates $m$ training datasets, each of size $N'$. The training samples in the new datasets are obtained by sampling from the original training set with replacement.

## Bagging to get dataset for base DTs

First, let us write a function to generate the training datasets for the base DTs with bagging.

In [314]:
from typing import List


def create_bags(
    train_input: np.array, train_label: np.array, num_bags: int, bag_size: int
) -> List[tuple]:
    """Applies bagging to generate num_bags datasets.

    Args:
        train_input: Inputs in the complete training dataset.
        train_label: Labels in the complete training dataset.
        num_bags: Number of datasets to create.
        bag_size: Size of the smaller datasets.

    Returns:
        List of tuples such that the tuple at index i contains the X, Y for bag i.
    """
    bags = []

    for i in range(num_bags):
        ...
        # Sample with replacement and add them to a bag.
        # ===== YOUR CODE HERE =====
        indices = np.random.choice(len(train_input), size=bag_size, replace=True)
        sample_input = train_input[indices]
        sample_label = train_label[indices]

        bags.append((sample_input, sample_label))
        # ==========================

    return bags

## Training the base DTs

Let us now write a function to train the base DTs.

1. First, use the `create_bags` function to create datasets for the base DTs. Note that the number of bags should be equal to the number of DTs.
1. Since we already have DTs implemented, pass the corresponding datasets to the `train` function to train $m$ of them and return the list of trained DTs.

In [315]:
chosen_metric = "entropy"

def train_base_dts(
    train_input: np.array, train_label: np.array, num_dts: int, bag_size: int
) -> List:
    """Trains num_dts decision trees with training dataset.

    Args:
        train_input: Inputs in the complete training dataset.
        train_label: Labels in the complete training dataset.
        num_dts: Number of decision trees.
        bag_size: Size of the datasets used to train each decision tree.

    Returns:
        List of trained decision trees.
    """
    base_dts = []

    bags = create_bags(train_input, train_label, num_dts, bag_size)
    for i in range(num_dts):
        ...
        # Train individual DTs with the `train` method from the decision tree problem and
        # add them to the base_dts list.
        # ===== YOUR CODE HERE =====
        bag = bags[i]
        tree = train(bag[0], bag[1], chosen_metric)
        base_dts.append(tree)
        # ==========================

    return base_dts

## Predictions via majority voting

Once we have trained the base decision trees, we can combine their individual predictions in different ways. For this assignment, let us implement majority voting. The idea here is to predict the class that is predicted by most number of decision trees (i.e., the mode of the $m$ predictions from base decision trees). If there are multiple modes, assume that ties are broken arbitrarily.

In [316]:
def predict_majority_voting(test_input: np.array, trained_base_dts: List) -> np.array:
    """Predicts the class labels based on majority voting.

    Args:
        test_input: Inputs in the test dataset.
        trained_base_dts: List of trained base decision trees.

    Returns:
        Array containing the predicted class labels for all test inputs.
    """
    # ===== YOUR CODE HERE =====
    predictions = []
    for dt in trained_base_dts:
        predictions.append(predict(test_input, dt))
    
    return [
        max(np.array(predictions)[:, i]) for i in range(len(test_input))
    ]
    # ==========================

## Analyzing the random forest predictions

Let us apply the two functions that we have written: `train_base_dts`, and `predict_majority_voting` to train a random forest on the training data from the apple quality dataset and make predictions on the test set (use `X_train`, `Y_train`, `X_test`, `Y_test` created in the decision tree problem).

Print the accuracy and F1 score on test set.

In [317]:
# Hyperparameters. You can try different values for these.
NUM_DTS = 5                   # Number of decision trees.
BAG_SIZE = len(X_train) // 2  # Size of the individual datasets for base decision trees.

# Call the train_base_dts to train a random forest.
# Call predict_majority_voting functions to get the predictions.
# Print the accuracy and F1 score (using either information gain or Gini impurity decrease
# as the quality metric).
# ===== YOUR CODE HERE =====
base_dts = train_base_dts(X_train, Y_train, NUM_DTS, BAG_SIZE)
Y_preds = predict_majority_voting(X_test, base_dts)
entropy_forest_statistics = get_statistics(Y_pred_entropy, Y_test, chosen_metric)

print(entropy_forest_statistics)
# ==========================

Information gain
  0.8183
  0.8239


We halve `BAG_SIZE` because we can train it much faster without significantly sacrificing precision and F1 score.

# Submission

Complete the required code and run ALL the code cells in this notebook. 
Submit the files mentioned in the README.